In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import zipfile

import matplotlib.pyplot as plt
import seaborn as sns
import random
import time

from sklearn.model_selection import train_test_split

#from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
import gc

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:
#Unzip the training dataset
local_zip = '../input/dogs-vs-cats/train.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')
#For Kaggle
#base_dir_zip = '/kaggle/working/'
#For peronal PC
base_dir_zip = '/kaggle/working/'

#Unzip all images
zip_ref.extractall(base_dir_zip)

zip_ref.close()

# Loop through all the files in the directory
train_dir = os.path.join(base_dir_zip, 'train')
train_img_names = os.listdir(train_dir)



In [ ]:
print(train_img_names[:10])

In [ ]:
categories= list()
yvalues = list()
for image in train_img_names:
    category = image.split(".")[0]
    if category == "dog":
        categories.append("dog")
        yvalues.append(1)
    else:
        categories.append("cat")
        yvalues.append(0)
df= pd.DataFrame({"Image":train_img_names, "Category": categories, "yvalue": yvalues})

In [ ]:
df["Category"].value_counts()

In [ ]:
train,validation= train_test_split(df, test_size=0.1)
train = train.reset_index(drop=True)
validation = validation.reset_index(drop=True)

In [ ]:
train["Category"].value_counts()

In [ ]:
validation["Category"].value_counts()

# MODEL 2

In [ ]:
size = 150
batch_size=128
#Black and white 1, color 3 (RGB)
colortype = 3


# All images will be rescaled by 1./255.
train_datagen = ImageDataGenerator( rescale = 1.0/255., horizontal_flip=True)


# --------------------
# Flow training images in batches of 20 using train_datagen generator
# --------------------
train_generator = train_datagen.flow_from_dataframe(train,
                                                    directory=train_dir,
                                                    x_col='Image',
                                                    y_col='Category',
                                                    batch_size=batch_size,
                                                    class_mode='binary',
                                                    target_size=(size, size))    


validation_datagen  = ImageDataGenerator( rescale = 1.0/255.)
validation_generator =  validation_datagen.flow_from_dataframe(validation,
                                                               directory=train_dir,
                                                               x_col='Image',
                                                               y_col='Category',
                                                               batch_size=batch_size,
                                                               class_mode  = 'binary',
                                                               target_size=(size, size))

model = Sequential()
model.add(Conv2D(filters=32, kernel_size=3,activation="relu", input_shape=(size,size,colortype)))
model.add(MaxPool2D(pool_size=2, strides=2))
model.add(Dropout(0.2))
model.add(Conv2D(filters=64, kernel_size=3,activation="relu"))
model.add(MaxPool2D(pool_size=2, strides=2))
model.add(Dropout(0.2))
model.add(Conv2D(filters=128, kernel_size=3, activation="relu"))
model.add(MaxPool2D(pool_size=2, strides=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(units=512, activation="relu"))
model.add(Dense(units=1, activation="sigmoid"))
model.summary()

In [ ]:
numepochs=1

early = EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True, verbose=1)
#checkpoint = ModelCheckpoint("best_model.hdf5", monitor='accuracy', verbose=1, save_best_only=True, mode='auto')

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Initial Training of the model...
startTime = time.time()

history=model.fit(train_generator, validation_data=validation_generator, epochs=numepochs,callbacks=[early])

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

# TEST RUN

In [ ]:
#Unzip the training dataset
local_zip = '../input/dogs-vs-cats/test1.zip'

zip_ref = zipfile.ZipFile(local_zip, 'r')
#For Kaggle
#base_dir_zip = '/kaggle/working/'
#For peronal PC
base_dir_zip = '/kaggle/working/'

#Unzip all images
zip_ref.extractall(base_dir_zip)

zip_ref.close()

# Loop through all the files in the directory
test_dir = os.path.join(base_dir_zip, 'test1')
test_img_names = os.listdir(test_dir)



test_df = pd.DataFrame({'Image': test_img_names})
test_df.head()



In [ ]:
test_df['id'] = test_df['Image'].apply(lambda x: x.rsplit(".", 1)[0])
test_df.head()

In [ ]:


test_datagen = ImageDataGenerator(rescale = 1.0/255.)
test_generator = test_datagen.flow_from_dataframe(test_df,
                                                 directory=test_dir,
                                                 x_col="Image",
                                                 y_col=None,
                                                 class_mode  = None,
                                                 target_size=(size,size),
                                                 batch_size=batch_size)

predictions = model.predict(test_generator)
predictions

In [ ]:
test_df["category"]=pd.DataFrame(predictions, columns=["category"])

def labelizor(prediction):
    if prediction > 0.5:
        return 1
    else:
        return 0
    
test_df["label"] = test_df["category"].apply(labelizor)
test_df

In [ ]:
import glob

files = glob.glob('/kaggle/working/train/*')
for f in files:
    os.remove(f)
    
files = glob.glob('/kaggle/working/test1/*')
for f in files:
    os.remove(f)

In [ ]:
submission = test_df[["id", "label"]]
submission

In [ ]:
submission.to_csv('submission.csv', index=False)